In [20]:
import collections
import networkx as nx
import sys
import os
sys.path.append(r'c:\\Users\\rotim\\OneDrive\\Documents\\Reading\\graph-code\\Skoltech-PhD-Thesis\\_codes\\SIR\\Influential-nodes-identification-in-complex-networks-via-information-entropy')
from algorithms import *
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import pickle

In [21]:
graph_collections, graph_name = read_graph("./test-graphs")


{'ABILENE': {'index': 0, 'nodes': 11, 'edges': 14}, 'ABVT': {'index': 1, 'nodes': 23, 'edges': 31}, 'AGIS': {'index': 2, 'nodes': 25, 'edges': 30}, 'ARN.GRAPHML': {'index': 3, 'nodes': 30, 'edges': 29}, 'BELLCANADA': {'index': 4, 'nodes': 48, 'edges': 64}, 'BELLSOUTH.GRAPHML': {'index': 5, 'nodes': 51, 'edges': 66}, 'BICS.GRAPHML': {'index': 6, 'nodes': 33, 'edges': 48}, 'CERNET.GRAPHML': {'index': 7, 'nodes': 41, 'edges': 58}, 'CESNET200706.GRAPHML': {'index': 8, 'nodes': 44, 'edges': 51}, 'CHINANET.GRAPHML': {'index': 9, 'nodes': 42, 'edges': 66}, 'COGENTCO.GRAPHML': {'index': 10, 'nodes': 197, 'edges': 243}, 'COLT.GRAPHML': {'index': 11, 'nodes': 153, 'edges': 177}, 'DELTACOM.GRAPHML': {'index': 12, 'nodes': 113, 'edges': 161}, 'GARR201110.GRAPHML': {'index': 13, 'nodes': 59, 'edges': 74}, 'GEANT2001': {'index': 14, 'nodes': 27, 'edges': 38}, 'GRNET.GRAPHML': {'index': 15, 'nodes': 37, 'edges': 42}, 'INTELLIFIBER.GRAPHML': {'index': 16, 'nodes': 73, 'edges': 95}, 'INTEROUTE.GRAPHML'

In [22]:
# India   145nodes 187edges #37
# US carrier 158nodes   189edges # 41
# Kentucky    754nodes   895edges # 20
# Albeni  # 0
# Geant  2001 #14
# Bell Canada # 4
# Abvt # 1
# Agis # 2
# 10, 18, 8
selection__= [37,41,20,0,14,4,1,2,10, 18, 8]
# selection__= []
# selection__=[i for i,j in enumerate(graph_name)]
def selection__from_graph(graph_coll,graph_name_list,sel_=[]):
    """_summary_

    Args:
        graph_coll (_type_): _description_
        sel_ (list, optional): _description_. Defaults to [].

    Returns:
        _type_: _description_
    """
    if sel_ !=[]:
        return [(j, graph_coll[k]) for i, j in enumerate(graph_name_list)
                for k in sel_ if i == k]
    else:
        return [(i, j) for i, j in zip(graph_name_list, graph_coll)]


In [23]:
G = graph_collections[37]
# geo_loc_data = get_geo_data("Internet2LatLong.csv")
geo_loc_data = {k: {"lat": v.get('Latitude', 0), "long": v.get('Longitude', 0)}
                for (k, v) in G.nodes(data=True)}
# print(geo_loc_data)
edge_geo_data_combined = assign_location(G, geo_loc_data)
# set edge attributes
set_edge_attr(G, edge_geo_data_combined)


In [24]:
gGrahps = selection__from_graph(graph_collections, graph_name, selection__)

print(gGrahps)

geo_loc_data_sample = {i: {k: {"lat": v.get('Latitude', 0), "long": v.get('Longitude', 0)}
     for (k, v) in g.nodes(data=True)} for i, g in gGrahps}
# print(geo_loc_data_sample)
edge_geo_data_sample = {k: assign_location(u, v) for (k, v), (j,u) in zip(geo_loc_data_sample.items(), gGrahps)}

[set_edge_attr(g[1], v[1]) for (v ,g) in zip(edge_geo_data_sample.items(), gGrahps)]

print()
# # edge_geo_data_combined_sample = {k: assign_location(g, v) for (k, v), g in zip(geo_loc_data_all.items(), graph_collections)}


[('ABILENE', <networkx.classes.graph.Graph object at 0x00000226C9737C40>), ('ABVT', <networkx.classes.graph.Graph object at 0x00000226C9780EB0>), ('AGIS', <networkx.classes.graph.Graph object at 0x00000226C99220A0>), ('BELLCANADA', <networkx.classes.graph.Graph object at 0x00000226C9922F10>), ('CESNET200706.GRAPHML', <networkx.classes.graph.Graph object at 0x00000226C9922070>), ('COGENTCO.GRAPHML', <networkx.classes.graph.Graph object at 0x00000226C98B0FD0>), ('GEANT2001', <networkx.classes.graph.Graph object at 0x00000226C98B0E50>), ('ION.GRAPHML', <networkx.classes.graph.Graph object at 0x00000226C9A72250>), ('KDL', <networkx.classes.graph.Graph object at 0x00000226C99228E0>), ('TATANLD.GRAPHML', <networkx.classes.graph.Graph object at 0x00000226C98997F0>), ('USCARRIER', <networkx.classes.graph.Graph object at 0x00000226C9899D90>)]



In [25]:
weight_sum =sum([sum(hub_information(G, i, 2))
     for (i, j) in sorted(list(G.degree()), key=lambda item: int(item[0]))])
node_strength_2_deg = [(i, sum(hub_information(G, i, 2)))  # /weight_sum
                       for (i, j) in sorted(list(G.degree()), key=lambda item: int(item[0]))]
degree__ = [(i, len(n_neighbor(G, i, 1))) for (i, j) in sorted(list(G.degree()),
                                                               key=lambda item: int(item[0]))]


In [26]:

source_1, source_2, source_3, source_4, *others = degree__, node_strength_2_deg, eccentricity(G),  approximate_current_flow_betweenness_centrality(G, weight="weight"), current_flow_betweenness_centrality(G), betweenness_centrality(G, weight="weight"),  closeness_centrality(G),      load_centrality(
    G),    #eigenvector_centrality(G),

# print(source_2, source_3)


In [27]:
sources_of_evidences = {
    "Degree": {g[0]: [(i, len(n_neighbor(g[1], i, 1))) for (i, j) in sorted(list(g[1].degree()), key=lambda item: int(item[0]))] for g in gGrahps},
    "Node Strength": {g[0]: [(i, sum(hub_information(g[1], i, 2))) for (i, j) in sorted(list(g[1].degree()), key=lambda item: int(item[0]))] for g in gGrahps},
    "Eccentricity": {g[0]: eccentricity(g[1]) for g in gGrahps},
    "Approx Curr. Flow Betwns Cent'y": {g[0]: approximate_current_flow_betweenness_centrality(g[1]) for g in gGrahps}
}
# sources_of_evidences["Degree"]["ABILENE"]

In [28]:
e_v = 2
varying_examples_multi(source_1, source_2, source_3, source_4,e_v)

[('70', 0.9986942454915201),
 ('52', 0.726920696302148),
 ('118', 0.6606933873804316),
 ('37', 0.04551598153655445)]

In [29]:
evis = [2,3,4]
selected_nodes_based_on_source ={i:{e: varying_examples_multi(sources_of_evidences["Degree"][i],
sources_of_evidences["Node Strength"][i], 
sources_of_evidences["Eccentricity"][i], 
    sources_of_evidences["Approx Curr. Flow Betwns Cent'y"][i], e) for e in evis} for i, j in gGrahps}

selected_nodes_based_on_source_ext = {k2:{k:len(v) for k,v in v2.items()} for k2,v2 in selected_nodes_based_on_source.items()}

# {i: { "#Nodes":  nx.number_of_nodes(j), "#Edges": nx.number_of_edges(j), "sel_":v} for (i,j), v in zip(gGrahps, selected_nodes_based_on_source.values())}
sel_node_summary={i: {"#Nodes":  nx.number_of_nodes(j), "#Edges": nx.number_of_edges(j), "Diameter":nx.diameter(j), "Density":nx.density(j)}for (i, j)in gGrahps }
sel_to_table = {
    i: (sel_node_summary[i] | selected_nodes_based_on_source_ext[i]) for i, j in gGrahps}
    


In [30]:
pd.DataFrame.from_dict(
    sel_to_table, orient='index')
# print()


,#Nodes,#Edges,Diameter,Density,2,3,4
ABILENE,11,14,5,0.254545,7,5,6
ABVT,23,31,7,0.122530,4,17,7
AGIS,25,30,7,0.100000,5,11,8
BELLCANADA,48,64,13,0.056738,8,36,8
CESNET200706.GRAPHML,44,51,6,0.053911,4,14,34
COGENTCO.GRAPHML,197,243,28,0.012587,5,156,11
GEANT2001,27,38,6,0.108262,5,8,17
ION.GRAPHML,125,146,25,0.018839,8,102,16
KDL,754,895,58,0.003153,16,486,41
TATANLD.GRAPHML,145,186,28,0.017816,4,122,7


In [31]:
selected_nodes_based_on_source["ABILENE"]
# {i:{varying_examples_multi(j,v2) for i,j in gGrahps for v2 in v} for v in selected_nodes_based_on_source.values()}
# varying_examples_multi(j)
# cluster_optimal_nodes()
# selected_nodes_based_on_source["BELLCANADA"][4]
# sel_error ={k2: {k: cluster_optimal_nodes(j, v) for k, v in v2.items() for i, j in gGrahps if {i, k2} } for k2, v2 in selected_nodes_based_on_source.items()}
clusters__ = {k2: {k: cluster_optimal_nodes(j, v) for k, v in v2.items()}  for (i, j), (k2, v2) in zip(tqdm(gGrahps), tqdm(selected_nodes_based_on_source.items()))}
clusters__


 91%|█████████ | 10/11 [00:17<00:01,  1.74s/it]


{'ABILENE': {2: [('7', set()),
   ('8', set()),
   ('6', {'3'}),
   ('9', {'0', '2'}),
   ('5', set()),
   ('4', set()),
   ('10', {'1'})],
  3: [('7', {'3', '4', '6'}),
   ('9', {'2'}),
   ('10', set()),
   ('8', {'5'}),
   ('1', {'0'})],
  4: [('7', {'10', '8'}),
   ('5', {'4'}),
   ('3', set()),
   ('2', {'0', '9'}),
   ('6', set()),
   ('1', set())]},
 'ABVT': {2: [('11', {'13', '15', '16', '17', '18', '21', '22'}),
   ('1', {'4', '6', '7', '8', '9'}),
   ('10', {'14', '19', '2', '20', '3'}),
   ('5', {'0', '12'})],
  3: [('1', {'4', '5'}),
   ('21', {'11'}),
   ('20', set()),
   ('9', set()),
   ('7', {'0'}),
   ('19', set()),
   ('14', set()),
   ('16', set()),
   ('15', set()),
   ('22', set()),
   ('17', set()),
   ('12', set()),
   ('13', set()),
   ('6', set()),
   ('10', {'2', '3'}),
   ('18', set()),
   ('8', set())],
  4: [('11', {'13', '14', '15', '16', '17', '18', '19', '21', '22'}),
   ('1', {'6', '8', '9'}),
   ('0', {'7'}),
   ('2', {'10', '20'}),
   ('4', set()),
   

In [32]:
# gGrahps[0][1]
cluster_optimal_nodes(gGrahps[0][1], selected_nodes_based_on_source["ABILENE"][2])


[('7', set()),
 ('8', set()),
 ('6', {'3'}),
 ('9', {'0', '2'}),
 ('5', set()),
 ('4', set()),
 ('10', {'1'})]

In [97]:
def cluster_optimal_nodes_test(G, opti_rank, b =1 ):
    # node_list = G.nodes()
    # opti_rank_nodes = [i for i, j in opti_rank]
    # print("opti_rank_nodes:", opti_rank_nodes)
    # remainder_nodes = set(node_list) - set(opti_rank_nodes)
    # print("remainder_nodes @ onset ...", remainder_nodes)
    # current_set_result_all = []

    # while remainder_nodes != set():
    #     print("remainder_nodes", remainder_nodes)
    #     chosen_list = [(i, n_neighbor(G, i, b)) for (i, j) in opti_rank]
    #     print("chosen_list", chosen_list)
    #     for x, y in chosen_list:
    #         chosen_set = set(y)            
    #         print("chosen_set", chosen_set)
    #         current_set = chosen_set.intersection(remainder_nodes)
    #         print("current_set", current_set)
    #         current_set_result = (x, current_set)
    #         print("current_set_result", current_set_result)
    #         current_set_result_all.append((current_set_result))
    #         print("current_set_result_all", current_set_result_all)
    #         remainder_nodes -= chosen_set
    #         print("remainder_nodes", remainder_nodes)
    #     b += 1
    node_list = G.nodes()
    print("Old Optimal nodes: opti_rank: ", opti_rank)
    opti_rank_nodes = [i for i, j in opti_rank]
    print("New Optimal nodes: opti_rank_nodes: ", opti_rank_nodes)
    remainder_nodes = set(node_list) - set(opti_rank_nodes)
    print("Remainder nodes:remainder_nodes: ", remainder_nodes)
    current_set_result_all =  []

    coll_ = collections.defaultdict(list)


    def rank_loop(G, opti_rank, b, remainder_nodes, current_set_result_all):
        while remainder_nodes != set():
            chosen_list = [(i, n_neighbor(G, i, b)) for (i, j) in opti_rank]
            for x, y in chosen_list:
                chosen_set = set(y)
                current_set = chosen_set.intersection(remainder_nodes)
                current_set_result = (x, current_set)
                current_set_result_all.append((current_set_result))
                remainder_nodes -= chosen_set
            b += 1

    rank_loop(G, opti_rank, b, remainder_nodes, current_set_result_all)
    # empty_controllers = set([i for i, j in current_set_result_all if j == set()])
    # # print(current_set_result_all)
    # print(empty_controllers)
    # print("Current Result: ", current_set_result_all)
    print("current_set_result_all: ", current_set_result_all)
    non_collated_current_set_result_all = [(i,k) for i,k in current_set_result_all if k!= set() ]
    print("non_collated_current_set. ",non_collated_current_set_result_all)
    # empty_controllers = set([i for i, j in current_set_result_all if j == set()])
    
    new_opti_rank = list(set([i for i, k in non_collated_current_set_result_all])) #if k == set()])
    new_opti_rank_2 = [(i, j) for k, j in opti_rank for i in new_opti_rank if i==k]
    empty_controllers = set(opti_rank_nodes) - set(new_opti_rank)
    print("Empty Controllers: ", empty_controllers)
    print("New_opti: ",new_opti_rank_2)
    rank_loop(G,new_opti_rank_2,1, empty_controllers, non_collated_current_set_result_all)
    for d, e in non_collated_current_set_result_all:
        coll_[d].extend(e)  # add to existing list or create a new one

    non_collated_current_set_result_all = list(coll_.items())
    # empty_controllers = set([i for i, j in current_set_result_all if j == []])
    # print("Empty Controllers: ", empty_controllers)
    # _ = [(i, j) for i, j in opti_rank if i not in empty_controllers ]
    # # [x for x in list_a if x[0] in list_b]
    # print("The remaining optimal Nodes:", _)
    # # print("current_set_result_all:", current_set_result_all)
    # # print("empty_controllers:", empty_controllers)
    # if empty_controllers != []:
    #     # print(empty_controllers)
    #     remainder_nodes = empty_controllers
    #     rank_loop(G, _, b, remainder_nodes, current_set_result_all)
    #     print("Current Result 2nd: ", current_set_result_all)        
    return [(i, set(j)) for i, j in non_collated_current_set_result_all]


In [100]:
cluster_optimal_nodes_test(gGrahps[0][1], selected_nodes_based_on_source["ABILENE"][3])


Old Optimal nodes: opti_rank:  [('7', 20.63709230769231), ('9', 4.899784087548063), ('10', 4.518380874484546), ('8', 2.2816482150926345), ('1', 0.04651503059323758)]
New Optimal nodes: opti_rank_nodes:  ['7', '9', '10', '8', '1']
Remainder nodes:remainder_nodes:  {'5', '4', '3', '6', '2', '0'}
current_set_result_all:  [('7', {'6'}), ('9', {'2'}), ('10', set()), ('8', {'5'}), ('1', {'0'}), ('7', {'3', '4'}), ('9', set()), ('10', set()), ('8', set()), ('1', set())]
non_collated_current_set.  [('7', {'6'}), ('9', {'2'}), ('8', {'5'}), ('1', {'0'}), ('7', {'3', '4'})]
Empty Controllers:  {'10'}
New_opti:  [('7', 20.63709230769231), ('9', 4.899784087548063), ('8', 2.2816482150926345), ('1', 0.04651503059323758)]


[('7', {'10', '3', '4', '6'}), ('9', {'2'}), ('8', {'5'}), ('1', {'0'})]

In [ ]:
cluster_optimal_nodes(gGrahps[0][1], selected_nodes_based_on_source["ABILENE"][2])


[('7', set()),
 ('8', set()),
 ('6', {'3'}),
 ('9', {'0', '2'}),
 ('5', set()),
 ('4', set()),
 ('10', {'1'})]